# `trading_index`

This tutorial shows all the ins and outs of the calendars' `trading_index` method. (For other calendar methods see the [calendar_methods.ipynb](./calendar_methods.ipynb) tutorial.)

The [What does it do?](#What-does-it-do?) section explains basic usage. The following sections then explore the method's arguments and options:

* [`intervals`](#intervals)
* [`period`](#period)
* [`closed`](#closed)
* [`force_close`, `force_break_close` and `force`](#force_close,-force_break_close-and-force)
* [`ignore_breaks`](#ignore_breaks)

The final section covers [overlapping indices](#Overlapping-indices).

In [2]:
# setup
import exchange_calendars as xcals
import pandas as pd

hkg = xcals.get_calendar("XHKG") # Hong Kong Stock Exchange
start, end = "2021-12-23", "2021-12-28"

In [3]:
# for reference
schedule = hkg.schedule.loc[start:end]
schedule

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 01:30:00,2021-12-23 04:00:00,2021-12-23 05:00:00,2021-12-23 08:00:00
2021-12-24 00:00:00+00:00,2021-12-24 01:30:00,NaT,NaT,2021-12-24 04:00:00
2021-12-28 00:00:00+00:00,2021-12-28 01:30:00,2021-12-28 04:00:00,2021-12-28 05:00:00,2021-12-28 08:00:00


### What does it do?

`trading_index` creates a trading index at a given period over a range of sessions.

In [4]:
hkg.trading_index(start, end, "30min", intervals=False)

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 05:00:00+00:00',
               '2021-12-23 05:30:00+00:00', '2021-12-23 06:00:00+00:00',
               '2021-12-23 06:30:00+00:00', '2021-12-23 07:00:00+00:00',
               '2021-12-23 07:30:00+00:00', '2021-12-24 01:30:00+00:00',
               '2021-12-24 02:00:00+00:00', '2021-12-24 02:30:00+00:00',
               '2021-12-24 03:00:00+00:00', '2021-12-24 03:30:00+00:00',
               '2021-12-28 01:30:00+00:00', '2021-12-28 02:00:00+00:00',
               '2021-12-28 02:30:00+00:00', '2021-12-28 03:00:00+00:00',
               '2021-12-28 03:30:00+00:00', '2021-12-28 05:00:00+00:00',
               '2021-12-28 05:30:00+00:00', '2021-12-28 06:00:00+00:00',
               '2021-12-28 06:30:00+00:00', '2021-12-28 07:00:00+00:00',
               '2021-12-28 07:30:00+00:00'],
      

Or over a single session, if `start` and `end` are passed as the same session.

In [5]:
hkg.trading_index(start=start, end=start, period="30min", intervals=False)

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 05:00:00+00:00',
               '2021-12-23 05:30:00+00:00', '2021-12-23 06:00:00+00:00',
               '2021-12-23 06:30:00+00:00', '2021-12-23 07:00:00+00:00',
               '2021-12-23 07:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### `intervals`

By passing `intervals` as False the index can be output as a `DatetimeIndex`, as shown above. Although by default the index is output as an `IntervalIndex`.

In [6]:
index = hkg.trading_index(start, start, "30min")
index

IntervalIndex([[2021-12-23 01:30:00, 2021-12-23 02:00:00), [2021-12-23 02:00:00, 2021-12-23 02:30:00), [2021-12-23 02:30:00, 2021-12-23 03:00:00), [2021-12-23 03:00:00, 2021-12-23 03:30:00), [2021-12-23 03:30:00, 2021-12-23 04:00:00) ... [2021-12-23 05:30:00, 2021-12-23 06:00:00), [2021-12-23 06:00:00, 2021-12-23 06:30:00), [2021-12-23 06:30:00, 2021-12-23 07:00:00), [2021-12-23 07:00:00, 2021-12-23 07:30:00), [2021-12-23 07:30:00, 2021-12-23 08:00:00)],
              closed='left',
              dtype='interval[datetime64[ns, UTC]]')

Let's make that a bit more readable by setting it to the index of a `DataFrame`, and have columns showing the index's left and right sides.

In [7]:
def show_as_df(index: pd.IntervalIndex) -> pd.DataFrame:
    index.name = "IntervalIndex"
    return pd.DataFrame(
        dict(left_side=index.left, right_side=index.right),
        index=index
    )

show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:00:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:00:00+00:00
"[2021-12-23 02:00:00, 2021-12-23 02:30:00)",2021-12-23 02:00:00+00:00,2021-12-23 02:30:00+00:00
"[2021-12-23 02:30:00, 2021-12-23 03:00:00)",2021-12-23 02:30:00+00:00,2021-12-23 03:00:00+00:00
"[2021-12-23 03:00:00, 2021-12-23 03:30:00)",2021-12-23 03:00:00+00:00,2021-12-23 03:30:00+00:00
"[2021-12-23 03:30:00, 2021-12-23 04:00:00)",2021-12-23 03:30:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 05:30:00)",2021-12-23 05:00:00+00:00,2021-12-23 05:30:00+00:00
"[2021-12-23 05:30:00, 2021-12-23 06:00:00)",2021-12-23 05:30:00+00:00,2021-12-23 06:00:00+00:00
"[2021-12-23 06:00:00, 2021-12-23 06:30:00)",2021-12-23 06:00:00+00:00,2021-12-23 06:30:00+00:00
"[2021-12-23 06:30:00, 2021-12-23 07:00:00)",2021-12-23 06:30:00+00:00,2021-12-23 07:00:00+00:00


### `period`

The trading index represents sessions and subsessions as a number of discrete periods of a given length.

When output as an `IntervalIndex` each indice represents a discrete period.

In [8]:
index[0]

Interval('2021-12-23 01:30:00', '2021-12-23 02:00:00', closed='left')

The length of each period of the example above is 30 minutes.

In [9]:
index.length

TimedeltaIndex(['0 days 00:30:00', '0 days 00:30:00', '0 days 00:30:00',
                '0 days 00:30:00', '0 days 00:30:00', '0 days 00:30:00',
                '0 days 00:30:00', '0 days 00:30:00', '0 days 00:30:00',
                '0 days 00:30:00', '0 days 00:30:00'],
               dtype='timedelta64[ns]', freq=None)

The period length is defined by the `period` argument which can be passed as the third positional argument or as a keyword argument. `period` can take either a `pd.Timedelta` or a string representing a pandas frequency. All the following are valid inputs representing a period length of 1 hour.

In [10]:
periods = [
    pd.Timedelta(hours=1),
    pd.Timedelta(1, "H"),
    pd.Timedelta(60, "min"),
    pd.Timedelta(60, "T"),
    "1H",
    "1h",
    "60T",
    "60min",
    "3600s"   
]

indexes = [ hkg.trading_index(start, start, period) for period in periods ]
for i, index in enumerate(indexes):
    pd.testing.assert_index_equal(index, indexes[i])

In [11]:
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:30:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:30:00+00:00
"[2021-12-23 02:30:00, 2021-12-23 03:30:00)",2021-12-23 02:30:00+00:00,2021-12-23 03:30:00+00:00
"[2021-12-23 03:30:00, 2021-12-23 04:30:00)",2021-12-23 03:30:00+00:00,2021-12-23 04:30:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:00:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:00:00+00:00
"[2021-12-23 06:00:00, 2021-12-23 07:00:00)",2021-12-23 06:00:00+00:00,2021-12-23 07:00:00+00:00
"[2021-12-23 07:00:00, 2021-12-23 08:00:00)",2021-12-23 07:00:00+00:00,2021-12-23 08:00:00+00:00


The maximum `period` is one day, in which case the trading index is returned as a `DatetimeIndex` of sessions.

In [12]:
hkg.trading_index(start, end, "1D")

DatetimeIndex(['2021-12-23 00:00:00+00:00', '2021-12-24 00:00:00+00:00',
               '2021-12-28 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='C')

At the other extreme, the period can be go all the way down to miliseconds to create a high frequency trading index.

In [13]:
high_freq = hkg.trading_index(start, start, "1ms")
high_freq

IntervalIndex([[2021-12-23 01:30:00, 2021-12-23 01:30:00.001000), [2021-12-23 01:30:00.001000, 2021-12-23 01:30:00.002000), [2021-12-23 01:30:00.002000, 2021-12-23 01:30:00.003000), [2021-12-23 01:30:00.003000, 2021-12-23 01:30:00.004000), [2021-12-23 01:30:00.004000, 2021-12-23 01:30:00.005000) ... [2021-12-23 07:59:59.995000, 2021-12-23 07:59:59.996000), [2021-12-23 07:59:59.996000, 2021-12-23 07:59:59.997000), [2021-12-23 07:59:59.997000, 2021-12-23 07:59:59.998000), [2021-12-23 07:59:59.998000, 2021-12-23 07:59:59.999000), [2021-12-23 07:59:59.999000, 2021-12-23 08:00:00)],
              closed='left',
              dtype='interval[datetime64[ns, UTC]]')

That's a lot of indices...

In [14]:
len(high_freq)

19800000

When the index is output as a `DatetimeIndex` the indices represent period bounds. Accordingly, the period length is given by the distance between consecutive indices. 

In [15]:
index = hkg.trading_index(start, start, "30min", intervals=False)

In [16]:
index[1:] - index[:-1]

TimedeltaIndex(['0 days 00:30:00', '0 days 00:30:00', '0 days 00:30:00',
                '0 days 00:30:00', '0 days 01:30:00', '0 days 00:30:00',
                '0 days 00:30:00', '0 days 00:30:00', '0 days 00:30:00',
                '0 days 00:30:00'],
               dtype='timedelta64[ns]', freq=None)

Huh? Why's there an hour and a half in there? Let's have a look at that index...

In [17]:
index

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 05:00:00+00:00',
               '2021-12-23 05:30:00+00:00', '2021-12-23 06:00:00+00:00',
               '2021-12-23 06:30:00+00:00', '2021-12-23 07:00:00+00:00',
               '2021-12-23 07:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [18]:
# recall
schedule.iloc[[0]]

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 01:30:00,2021-12-23 04:00:00,2021-12-23 05:00:00,2021-12-23 08:00:00


Hong Kong has a lunch break. The trading index covers the morning subsession starting with the open, and then the afternoon subsession starting with the break-end. The hour and a half represents the difference between the last indice of the morning subsession and the first indice of the afternoon subsession. (See [`ignore_breaks`](#ignore_breaks) to treat session as continuous.)

Ok, fine, but why hasn't the index included indices for 04:00 or 08:00? Because, it's `closed` on the "left" (by default)...

### `closed`

The `closed` parameter has a different effect depending on whether the index is being output as an `IntervalIndex` (default) or `DatetimeIndex` (i.e. if `intervals=False`).

If the index is being output as an `IntervalIndex` then `closed` can take either "left" (default) or "right". This value is simply passed through to the `closed` parameter of the `IntervalIndex` (to define the side on which all the intervals should be closed). In this case `closed` has no other effect.

If the index is being output as an `DatetimeIndex` then `closed` can take either "left", "right", "both" or "neither". In this case `closed` determines **the side of each implied period that an indice should be defined on**. The consequence of each value is best understood by considering whether or not the left side of the first implied period and/or the right side of last implied period are included as indices:
* "left" (default) - include left side of first period, do not include right side of last period.
* "right" - do not include left side of first period, include right side of last period.
* "both" - include both left side of first period and right side of last period.
* "neither" - do not include either left side of first period or right side of last period.

So, going back to the prior example, passing `closed` as right will include the 04:00 and 08:00 indices.

In [19]:
hkg.trading_index(start, start, "30min", closed="right", intervals=False)

DatetimeIndex(['2021-12-23 02:00:00+00:00', '2021-12-23 02:30:00+00:00',
               '2021-12-23 03:00:00+00:00', '2021-12-23 03:30:00+00:00',
               '2021-12-23 04:00:00+00:00', '2021-12-23 05:30:00+00:00',
               '2021-12-23 06:00:00+00:00', '2021-12-23 06:30:00+00:00',
               '2021-12-23 07:00:00+00:00', '2021-12-23 07:30:00+00:00',
               '2021-12-23 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Although now that only the right sides of the implied periods are being defined - the open (01:30) and start of the afternoon subsession (05:00) are not included...

`closed` can be passed as "both" to include both the left and the right side of each implied interval.

In [20]:
hkg.trading_index(start, start, "30min", closed="both", intervals=False)

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 04:00:00+00:00',
               '2021-12-23 05:00:00+00:00', '2021-12-23 05:30:00+00:00',
               '2021-12-23 06:00:00+00:00', '2021-12-23 06:30:00+00:00',
               '2021-12-23 07:00:00+00:00', '2021-12-23 07:30:00+00:00',
               '2021-12-23 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

And if you have a use case where you don't require either the left side of a (sub)session's first period or the right side of its last one, then you're after "neither".

In [21]:
hkg.trading_index(start, start, "30min", closed="neither", intervals=False)

DatetimeIndex(['2021-12-23 02:00:00+00:00', '2021-12-23 02:30:00+00:00',
               '2021-12-23 03:00:00+00:00', '2021-12-23 03:30:00+00:00',
               '2021-12-23 05:30:00+00:00', '2021-12-23 06:00:00+00:00',
               '2021-12-23 06:30:00+00:00', '2021-12-23 07:00:00+00:00',
               '2021-12-23 07:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### `force_close`, `force_break_close` and `force`

Have a look at this.

In [22]:
index = hkg.trading_index(start, start, "80T")
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:50:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:50:00+00:00
"[2021-12-23 02:50:00, 2021-12-23 04:10:00)",2021-12-23 02:50:00+00:00,2021-12-23 04:10:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:20:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:20:00+00:00
"[2021-12-23 06:20:00, 2021-12-23 07:40:00)",2021-12-23 06:20:00+00:00,2021-12-23 07:40:00+00:00
"[2021-12-23 07:40:00, 2021-12-23 09:00:00)",2021-12-23 07:40:00+00:00,2021-12-23 09:00:00+00:00


Specifically, consider the last indice of the morning subsession against the break-start time...

In [23]:
# for reference
schedule.iloc[[0]]

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 01:30:00,2021-12-23 04:00:00,2021-12-23 05:00:00,2021-12-23 08:00:00


In [24]:
index[1]

Interval('2021-12-23 02:50:00', '2021-12-23 04:10:00', closed='left')

It can be seen that whilst this indice represents, like all the others, a period of 80 minutes, only the first 70 minutes of these actually represent a trading period, with the last 10 minutes, from 04:00 to 04:10, falling within the subsequent break.

Similarly, it can be see that the last indice includes a full hour (08:00 - 09:00) that falls after the close.

**The final indice of a (sub)session will always include a non-trading period whenever the period length is not a factor of the (sub)session duration.**

The `force_close` and `force_break_close` options are provided to force the right side of the final period to, respectively, the session / morning subsession close.

In [25]:
index = hkg.trading_index(
    start, start, "80T", force_close=True, force_break_close=True
)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:50:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:50:00+00:00
"[2021-12-23 02:50:00, 2021-12-23 04:00:00)",2021-12-23 02:50:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:20:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:20:00+00:00
"[2021-12-23 06:20:00, 2021-12-23 07:40:00)",2021-12-23 06:20:00+00:00,2021-12-23 07:40:00+00:00
"[2021-12-23 07:40:00, 2021-12-23 08:00:00)",2021-12-23 07:40:00+00:00,2021-12-23 08:00:00+00:00


Note that these options can be used independently.

In [26]:
index = hkg.trading_index(start, start, "80T", force_break_close=True)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:50:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:50:00+00:00
"[2021-12-23 02:50:00, 2021-12-23 04:00:00)",2021-12-23 02:50:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:20:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:20:00+00:00
"[2021-12-23 06:20:00, 2021-12-23 07:40:00)",2021-12-23 06:20:00+00:00,2021-12-23 07:40:00+00:00
"[2021-12-23 07:40:00, 2021-12-23 09:00:00)",2021-12-23 07:40:00+00:00,2021-12-23 09:00:00+00:00


If you are going to set both `force_close` and `force_break_close` to the same value then you're better off passing just the `force` convenience option. If passed, `force` sets both `force_close` and `force_break_close` (anything passed to either will be overriden).

In [27]:
index = hkg.trading_index(start, start, "80T", force=True)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:50:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:50:00+00:00
"[2021-12-23 02:50:00, 2021-12-23 04:00:00)",2021-12-23 02:50:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:20:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:20:00+00:00
"[2021-12-23 06:20:00, 2021-12-23 07:40:00)",2021-12-23 06:20:00+00:00,2021-12-23 07:40:00+00:00
"[2021-12-23 07:40:00, 2021-12-23 08:00:00)",2021-12-23 07:40:00+00:00,2021-12-23 08:00:00+00:00


An obvious consequence of forcing the close is that any indice that's forced will have a shorter period length than `period`.

In [28]:
index.length

TimedeltaIndex(['0 days 01:20:00', '0 days 01:10:00', '0 days 01:20:00',
                '0 days 01:20:00', '0 days 00:20:00'],
               dtype='timedelta64[ns]', freq=None)

**If having indices that represent only trading periods is more important to you than having them all relfect the same period length, then forcing the close is the way to go.**

The force close options can also be employed when creating `DatetimeIndex`.

When `closed` is either "right" or "both" the effect will be similar to that for the `IntervalIndex`, in that if the (sub)sessions' last indices otherwise fall later than the close they will be curtailed to the close.

In [29]:
# NB forcing only the session close, not the break
hkg.trading_index(start, start, "80T", closed="right", force_close=True, intervals=False)

DatetimeIndex(['2021-12-23 02:50:00+00:00', '2021-12-23 04:10:00+00:00',
               '2021-12-23 06:20:00+00:00', '2021-12-23 07:40:00+00:00',
               '2021-12-23 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

When `closed` is either "left" or "neither" the effect of forcing the close simply adds the corresponding close to the index. This might be thought of as adding the right side of the (sub)session's last implied period, albeit curtailed to the close. 

Recalling the question of why 04:00 and 08:00 aren't included when doing this.

In [30]:
hkg.trading_index(start, start, "30min", intervals=False)

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 05:00:00+00:00',
               '2021-12-23 05:30:00+00:00', '2021-12-23 06:00:00+00:00',
               '2021-12-23 06:30:00+00:00', '2021-12-23 07:00:00+00:00',
               '2021-12-23 07:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Forcing the closes offers a way of including them if that's what's required.

In [31]:
hkg.trading_index(start, start, "30min", force=True, intervals=False)

DatetimeIndex(['2021-12-23 01:30:00+00:00', '2021-12-23 02:00:00+00:00',
               '2021-12-23 02:30:00+00:00', '2021-12-23 03:00:00+00:00',
               '2021-12-23 03:30:00+00:00', '2021-12-23 04:00:00+00:00',
               '2021-12-23 05:00:00+00:00', '2021-12-23 05:30:00+00:00',
               '2021-12-23 06:00:00+00:00', '2021-12-23 06:30:00+00:00',
               '2021-12-23 07:00:00+00:00', '2021-12-23 07:30:00+00:00',
               '2021-12-23 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Worth nothing that if all closes are forced then the same index will be returned by passing `closed` as:
- "left" or "both".
- "right" or "neither".

### `ignore_breaks`

`ignore_breaks` provides for ignoring any session breaks and instead treating every session as if it were continuous. The following shows how by default `ignore_breaks` is False such that non-trading indices are not introduced within breaks.

In [32]:
# for reference
schedule.iloc[[0]]

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 01:30:00,2021-12-23 04:00:00,2021-12-23 05:00:00,2021-12-23 08:00:00


In [33]:
index = hkg.trading_index(start, start, "30T")
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:00:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:00:00+00:00
"[2021-12-23 02:00:00, 2021-12-23 02:30:00)",2021-12-23 02:00:00+00:00,2021-12-23 02:30:00+00:00
"[2021-12-23 02:30:00, 2021-12-23 03:00:00)",2021-12-23 02:30:00+00:00,2021-12-23 03:00:00+00:00
"[2021-12-23 03:00:00, 2021-12-23 03:30:00)",2021-12-23 03:00:00+00:00,2021-12-23 03:30:00+00:00
"[2021-12-23 03:30:00, 2021-12-23 04:00:00)",2021-12-23 03:30:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 05:30:00)",2021-12-23 05:00:00+00:00,2021-12-23 05:30:00+00:00
"[2021-12-23 05:30:00, 2021-12-23 06:00:00)",2021-12-23 05:30:00+00:00,2021-12-23 06:00:00+00:00
"[2021-12-23 06:00:00, 2021-12-23 06:30:00)",2021-12-23 06:00:00+00:00,2021-12-23 06:30:00+00:00
"[2021-12-23 06:30:00, 2021-12-23 07:00:00)",2021-12-23 06:30:00+00:00,2021-12-23 07:00:00+00:00


Passing `ignore_breaks` as True will include indices through any break.

In [34]:
index = hkg.trading_index(start, start, "30T", ignore_breaks=True)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:00:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:00:00+00:00
"[2021-12-23 02:00:00, 2021-12-23 02:30:00)",2021-12-23 02:00:00+00:00,2021-12-23 02:30:00+00:00
"[2021-12-23 02:30:00, 2021-12-23 03:00:00)",2021-12-23 02:30:00+00:00,2021-12-23 03:00:00+00:00
"[2021-12-23 03:00:00, 2021-12-23 03:30:00)",2021-12-23 03:00:00+00:00,2021-12-23 03:30:00+00:00
"[2021-12-23 03:30:00, 2021-12-23 04:00:00)",2021-12-23 03:30:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 04:00:00, 2021-12-23 04:30:00)",2021-12-23 04:00:00+00:00,2021-12-23 04:30:00+00:00
"[2021-12-23 04:30:00, 2021-12-23 05:00:00)",2021-12-23 04:30:00+00:00,2021-12-23 05:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 05:30:00)",2021-12-23 05:00:00+00:00,2021-12-23 05:30:00+00:00
"[2021-12-23 05:30:00, 2021-12-23 06:00:00)",2021-12-23 05:30:00+00:00,2021-12-23 06:00:00+00:00


In the above example the start of the afternoon session (05.00) is preserved as an indice only because it falls 'on frequency'. This won't be the case for all periods.

In [35]:
index = hkg.trading_index(start, start, "1H", ignore_breaks=True)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 02:30:00)",2021-12-23 01:30:00+00:00,2021-12-23 02:30:00+00:00
"[2021-12-23 02:30:00, 2021-12-23 03:30:00)",2021-12-23 02:30:00+00:00,2021-12-23 03:30:00+00:00
"[2021-12-23 03:30:00, 2021-12-23 04:30:00)",2021-12-23 03:30:00+00:00,2021-12-23 04:30:00+00:00
"[2021-12-23 04:30:00, 2021-12-23 05:30:00)",2021-12-23 04:30:00+00:00,2021-12-23 05:30:00+00:00
"[2021-12-23 05:30:00, 2021-12-23 06:30:00)",2021-12-23 05:30:00+00:00,2021-12-23 06:30:00+00:00
"[2021-12-23 06:30:00, 2021-12-23 07:30:00)",2021-12-23 06:30:00+00:00,2021-12-23 07:30:00+00:00
"[2021-12-23 07:30:00, 2021-12-23 08:30:00)",2021-12-23 07:30:00+00:00,2021-12-23 08:30:00+00:00


### `Overlapping indices`

Now, if you've been playing with `trading_index` as you've worked through this tutorial and you've managed to get this far without raising an error, then you just haven't been trying hard enough...

In [36]:
period_105 = hkg.trading_index(start, start, "105T")
period_105

IntervalIndex([[2021-12-23 01:30:00, 2021-12-23 03:15:00), [2021-12-23 03:15:00, 2021-12-23 05:00:00), [2021-12-23 05:00:00, 2021-12-23 06:45:00), [2021-12-23 06:45:00, 2021-12-23 08:30:00)],
              closed='left',
              dtype='interval[datetime64[ns, UTC]]')

Just one more minute...

In [ ]:
period_106 = hkg.trading_index(start, start, "106T")
# Run call for full traceback

```python
---------------------------------------------------------------------------
IntervalsOverlapError                     Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_16260/3133521167.py in <module>
----> 1 period_106 = hkg.trading_index(start, start, "106T")

IntervalsOverlapError: Unable to create trading index as intervals would overlap. This can occur when the frequency is longer than a break or the gap between one session's close and the next session's open. To shorten intervals that would otherwise overlap either pass `curtail_overlaps` as True or pass `force_close` and/or `force_break_close` as True.
```

What happened? The error message pretty much explains it. Lets have a better look at the index with the 105 minutes period.

In [38]:
show_as_df(period_105)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 03:15:00)",2021-12-23 01:30:00+00:00,2021-12-23 03:15:00+00:00
"[2021-12-23 03:15:00, 2021-12-23 05:00:00)",2021-12-23 03:15:00+00:00,2021-12-23 05:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:45:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:45:00+00:00
"[2021-12-23 06:45:00, 2021-12-23 08:30:00)",2021-12-23 06:45:00+00:00,2021-12-23 08:30:00+00:00


See how the right side of the second indice (the last indice of the morning subsession) has bumped right up against the left side of the third indice (the first indice of the afternoon subsession). The indices don't overlap because all the intervals are all `closed` on the "left" (by default), so the exact instance '2021-12-23 05:00:00' is present in the third indice...

In [39]:
pd.Timestamp('2021-12-23 05:00:00', tz="UTC") in period_105[2]

True

...but not the second.

In [40]:
pd.Timestamp('2021-12-23 05:00:00', tz="UTC") in period_105[1]

False

However, add just one more minute to the period length and these indices overlap, hence the `IntervalsOverlapError`. As noted in the error message, the overlap can be prevented by forcing close on the break.

In [41]:
show_as_df(hkg.trading_index(start, start, "106T", force_break_close=True))

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 03:16:00)",2021-12-23 01:30:00+00:00,2021-12-23 03:16:00+00:00
"[2021-12-23 03:16:00, 2021-12-23 04:00:00)",2021-12-23 03:16:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:46:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:46:00+00:00
"[2021-12-23 06:46:00, 2021-12-23 08:32:00)",2021-12-23 06:46:00+00:00,2021-12-23 08:32:00+00:00


Or, `curtail_overlaps` can be passed as True to curtail overlapping indices. This has the effect of curtailing the right side of the earlier of the overlapping indices to the left side of the latter.

In [42]:
index = hkg.trading_index(start, start, "111T", curtail_overlaps=True)
show_as_df(index)

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 03:21:00)",2021-12-23 01:30:00+00:00,2021-12-23 03:21:00+00:00
"[2021-12-23 03:21:00, 2021-12-23 05:00:00)",2021-12-23 03:21:00+00:00,2021-12-23 05:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 06:51:00)",2021-12-23 05:00:00+00:00,2021-12-23 06:51:00+00:00
"[2021-12-23 06:51:00, 2021-12-23 08:42:00)",2021-12-23 06:51:00+00:00,2021-12-23 08:42:00+00:00


Obviously this has the same effect as forcing the close in so much as the lengths of some indices will now be less than the passed `period`.

In [43]:
index.length

TimedeltaIndex(['0 days 01:51:00', '0 days 01:39:00', '0 days 01:51:00',
                '0 days 01:51:00'],
               dtype='timedelta64[ns]', freq=None)

**NB** that `curtail_overlaps` is only available if outputing as an `IntervalIndex` (if `intervals=False` then `curtail_overlaps` has no effect).

Another option, not noted by the error message, can be to _increase_ the period. 

In [44]:
show_as_df(hkg.trading_index(start, start, "150T"))

,left_side,right_side
IntervalIndex,,
"[2021-12-23 01:30:00, 2021-12-23 04:00:00)",2021-12-23 01:30:00+00:00,2021-12-23 04:00:00+00:00
"[2021-12-23 05:00:00, 2021-12-23 07:30:00)",2021-12-23 05:00:00+00:00,2021-12-23 07:30:00+00:00
"[2021-12-23 07:30:00, 2021-12-23 10:00:00)",2021-12-23 07:30:00+00:00,2021-12-23 10:00:00+00:00


(I'll leave it to you to work out what happened there.)

The possibility for overlaps are not limited to calendars with lunch breaks. Overlaps can also occur when the right side of the last indice of one session falls later than the left side of the first indice of the next session. For a 24 hour calendar, like CMES, this will occur whenever the period is not a factor of 24 hours.

In [45]:
# grab a 24-hour calendar
cmes = xcals.get_calendar("CMES")
# for reference
start24, end24 = "2021-12-01", "2021-12-03"
cmes.schedule[start24:end24]

,market_open,break_start,break_end,market_close
2021-12-01 00:00:00+00:00,2021-11-30 23:00:00,NaT,NaT,2021-12-01 23:00:00
2021-12-02 00:00:00+00:00,2021-12-01 23:00:00,NaT,NaT,2021-12-02 23:00:00
2021-12-03 00:00:00+00:00,2021-12-02 23:00:00,NaT,NaT,2021-12-03 23:00:00


8 hours is a factor of 24 hours...

In [46]:
show_as_df(cmes.trading_index(start24, end24, "8H"))

,left_side,right_side
IntervalIndex,,
"[2021-11-30 23:00:00, 2021-12-01 07:00:00)",2021-11-30 23:00:00+00:00,2021-12-01 07:00:00+00:00
"[2021-12-01 07:00:00, 2021-12-01 15:00:00)",2021-12-01 07:00:00+00:00,2021-12-01 15:00:00+00:00
"[2021-12-01 15:00:00, 2021-12-01 23:00:00)",2021-12-01 15:00:00+00:00,2021-12-01 23:00:00+00:00
"[2021-12-01 23:00:00, 2021-12-02 07:00:00)",2021-12-01 23:00:00+00:00,2021-12-02 07:00:00+00:00
"[2021-12-02 07:00:00, 2021-12-02 15:00:00)",2021-12-02 07:00:00+00:00,2021-12-02 15:00:00+00:00
"[2021-12-02 15:00:00, 2021-12-02 23:00:00)",2021-12-02 15:00:00+00:00,2021-12-02 23:00:00+00:00
"[2021-12-02 23:00:00, 2021-12-03 07:00:00)",2021-12-02 23:00:00+00:00,2021-12-03 07:00:00+00:00
"[2021-12-03 07:00:00, 2021-12-03 15:00:00)",2021-12-03 07:00:00+00:00,2021-12-03 15:00:00+00:00
"[2021-12-03 15:00:00, 2021-12-03 23:00:00)",2021-12-03 15:00:00+00:00,2021-12-03 23:00:00+00:00


but 7 minutes is not...

In [ ]:
cmes.trading_index(start24, end24, "7T")
# Run cell for full traceback

```python
---------------------------------------------------------------------------
IntervalsOverlapError                     Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_16260/821017084.py in <module>
----> 1 cmes.trading_index(start24, end24, "7T")

IntervalsOverlapError: Unable to create trading index as intervals would overlap. This can occur when the frequency is longer than a break or the gap between one session's close and the next session's open. To shorten intervals that would otherwise overlap either pass `curtail_overlaps` as True or pass `force_close` and/or `force_break_close` as True.
```

In [48]:
show_as_df(cmes.trading_index(start, end, "7T", force_close=True))

,left_side,right_side
IntervalIndex,,
"[2021-12-22 23:00:00, 2021-12-22 23:07:00)",2021-12-22 23:00:00+00:00,2021-12-22 23:07:00+00:00
"[2021-12-22 23:07:00, 2021-12-22 23:14:00)",2021-12-22 23:07:00+00:00,2021-12-22 23:14:00+00:00
"[2021-12-22 23:14:00, 2021-12-22 23:21:00)",2021-12-22 23:14:00+00:00,2021-12-22 23:21:00+00:00
"[2021-12-22 23:21:00, 2021-12-22 23:28:00)",2021-12-22 23:21:00+00:00,2021-12-22 23:28:00+00:00
"[2021-12-22 23:28:00, 2021-12-22 23:35:00)",2021-12-22 23:28:00+00:00,2021-12-22 23:35:00+00:00
...,...,...
"[2021-12-28 22:27:00, 2021-12-28 22:34:00)",2021-12-28 22:27:00+00:00,2021-12-28 22:34:00+00:00
"[2021-12-28 22:34:00, 2021-12-28 22:41:00)",2021-12-28 22:34:00+00:00,2021-12-28 22:41:00+00:00
"[2021-12-28 22:41:00, 2021-12-28 22:48:00)",2021-12-28 22:41:00+00:00,2021-12-28 22:48:00+00:00


Overlapping indices are also a consideration for trading indexes output as `DatetimeIndex`, although only if the index is being `closed` on either the "right" side or "both" (it's not possible for a `DatetimeIndex` `closed` on the "left" side or "neither" to have overlapping indices as the right side is not defined, and hence cannot overlap the left side of the following indice).

If the index is being `closed` on the "right" then the periods represented by the indices will overlap whenever they would overlap for the equivalent `IntervalIndex`. Only difference is that an `IndicesOverlapError` is raised rather than an `IntervalsOverlapError`.

In [ ]:
hkg.trading_index(start, start, "106T", closed="right", intervals=False)

```python
---------------------------------------------------------------------------
IndicesOverlapError                       Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_16260/3132062204.py in <module>
----> 1 hkg.trading_index(start, start, "106T", closed="right", intervals=False)

IndicesOverlapError: Unable to create trading index as an indice would fall to the right of (later than) the subsequent indice. This can occur when the frequency is longer than a break or the frequency is longer than the gap between one session's close and the next session's open. Consider  passing `closed` as `left` or passing `force_close` and/or `force_break_close` as True.
```

If the trading index is being closed on "both" sides then the overlap will occur one minute earlier, as whilst this is on the limit with `closed` "right"...

In [50]:
hkg.trading_index(start, start, "105T", closed="right", intervals=False)

DatetimeIndex(['2021-12-23 03:15:00+00:00', '2021-12-23 05:00:00+00:00',
               '2021-12-23 06:45:00+00:00', '2021-12-23 08:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

It's over the limit if `closed` "both". This is because here 05:00 would be a minute of both the last implied period of the morning subsession and the first implied period of the afternoon subsession.

In [ ]:
hkg.trading_index(start, start, "105T", closed="both", intervals=False)

```python
---------------------------------------------------------------------------
IndicesOverlapError                       Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_16260/377205800.py in <module>
----> 1 hkg.trading_index(start, start, "105T", closed="both", intervals=False)

IndicesOverlapError: Unable to create trading index as an indice would fall to the right of (later than) the subsequent indice. This can occur when the frequency is longer than a break or the frequency is longer than the gap between one session's close and the next session's open. Consider  passing `closed` as `left` or passing `force_close` and/or `force_break_close` as True.
```

As with an `IntervalIndex`, overlapping indices can be prevented by forcing the applicable close(s). Unlike `IntervalIndex`, `curtail_indices` is **not** available for `DatetimeIndex`.

Although `curtail_overlaps` is unavailable, a similar effect can be achieved by passing `closed` as "left" instead of "both" or "neither" instead of "right" (the effect isn't exactly the same as `curtail_overlaps` allows for curtailing only overlapping indices, whilst changing closed to "left" or "neither" has the effect of losing the right side of the last implied period of _all_ sessions and subsessions, regardless of whether they would overlap or not).